In [ ]:
import getpass

user = getpass.getuser()
password = getpass.getpass("Password: ")
host = input("Host: ")
database = input("Database: ")

In [14]:
import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

connection = psycopg2.connect(user = user,
        password = password,
        host = host,
        port = 5432,
        database = database)
cur = connection.cursor()

symbol = "aal"

ticker = yf.Ticker(symbol)

history = ticker.history(period="Max")

history.columns = ["open", "high", "low", "close", "volume", "dividends", "stock_splits"]
history.insert(0, "symbol", symbol)

string_engine = engine = "postgresql://" + user + ":" + password + "@" + host + ":5432/" + database
engine = create_engine(string_engine)

history.to_sql('stocks', con=engine, if_exists='append', index = True, schema='machine_learning', index_label="date")

delete_dupes = cur.execute("DELETE FROM machine_learning.stocks a USING(SELECT MIN(ctid) as ctid, date, symbol from machine_learning.stocks GROUP BY date, symbol HAVING COUNT(*) > 1) b WHERE a.date = b.date AND a.ctid <> b.ctid AND a.symbol = b.symbol")
connection.commit()
connection.close()
print("Run was a Success!")                     

Run was a Success!
